## 03a - Assess Outlink ILLs Through time
Of all of the outlinks throughout time this calculates the current ILL situation
note: you cannot get the historical ILL sitution so this is whether or not it CURRENTLY has an ILL

import:

    00_en_as_rev_outlinks_m.pkl
    00_ar_as_rev_outlinks_m.pkl
    
output:
    ILLdf_en.pkl
    
    
The Plan:
- it grabs all of the outlinks in the english and arabic versions of the arab spring throughout time
- organize them by ones that are only in each language. 
- identify ones that are in both by if there were english outlinks which is now done by functions
- that had an arabic ILL and if that arabic ILL was also an outlink
- make a table with all of this information

1. Get Data
2. Make List of all the Unique Outlinks for Comparison
3. Get the ILLs and sort them

In [1]:
import numpy as np
import pandas as pd
import wikifunctions as wf #Brian's stuff
from urllib.parse import unquote
import pickle

## 1- Get Data

### English

In [2]:
#open the English revision outlinks with a revision id

with open('../Data/00_en_as_rev_outlinks_m.pkl', 'rb') as f:
    data = pickle.load(f)
    
result_series = pd.Series(data)

In [3]:
# result_series.keys

In [4]:
#get rid of duplicates in each outlinks list
#english
outlinksUnique = {}
keys = result_series.keys()

for i in keys:
    outlinksUnique[i] = np.unique(result_series[i]).tolist()

In [5]:
# outlinksUnique.keys()

### Arabic

In [6]:
#open the English revision outlinks with a revision id

with open('../Data/00_ar_as_rev_outlinks_m.pkl', 'rb') as f:
    ar_data = pickle.load(f)
    
ar_result_series = pd.Series(ar_data)

In [7]:
# ar_result_series

In [8]:
#arabic
ar_outlinksUnique = {}
ar_keys = ar_result_series.keys()

for i in ar_keys:
    ar_outlinksUnique[i] = np.unique(ar_result_series[i]).tolist()

In [9]:
# ar_outlinksUnique.keys()

## 2 - Make List of all the Unique Outlinks for Comparison

In [10]:
import itertools

def merge(*lists):
    newlist = []
    for i in lists:
            newlist.extend(i)
    return newlist

### English

In [11]:
everyOutlink = merge(outlinksUnique.values()) #makes lists of lists
everyOutlink = list(itertools.chain.from_iterable(everyOutlink)) #puts them all in one list
everyOutlink = np.unique(everyOutlink).tolist() #a list of alphabetized unique outlinks

len(everyOutlink)

848

In [12]:
ar_everyOutlink = merge(ar_outlinksUnique.values()) #makes lists of lists
ar_everyOutlink = list(itertools.chain.from_iterable(ar_everyOutlink)) #puts them all in one list
ar_everyOutlink = np.unique(ar_everyOutlink).tolist() #a list of alphabetized unique outlinks

len(ar_everyOutlink)

402

## 3- Get ILLs

### English

In [13]:
#get ILLs
onlyEnNoILL = [] #a list of english links - this is an En ILL but no arabic ILL 
enOutarILL = [] #a list of english links - there is an arILL in out_ar_lyst_clean
onlyEn = [] #a list of english links- that have an arabic out link but it is not in in out_ar_lyst_clean

make functions

In [14]:
#gets all of the arabic ILL for english outlinks
#first in English, second in Arabic

def seeIfnoILL(out_en_lyst_clean):
    #if there is one it is added to:
    arILLforEnOutlinks = []

    #if there isn't one it is added to: 
    onlyEnNoILL = [] 

    for enOut in out_en_lyst_clean:
        ILL = wf.get_interlanguage_links(enOut)
        try:
            arILL = ILL['ar']
            arILLforEnOutlinks.append(arILL)
            #print(arILL)
        except KeyError as ke:
            #print(enOut)
            onlyEnNoILL.append(enOut) #no arabic ILL 
            
    return onlyEnNoILL, arILLforEnOutlinks

In [15]:
#checks to see if the arabic ILLs are in the outlinks of the arabic page
#returns things in English

def seeRelatedtoAR(arILLforEnOutlinks, out_ar_lyst_clean):
    #if they are add it to: 
    enOutarILL = []
    enOutarILL_arabic = []

    #if the arabic ILL is not referenced also in the arabic page add it to
    onlyEn = []
    onlyEn_english = []


    for arOut in arILLforEnOutlinks: #goes through all of the arabic outlinks
        ILL = wf.get_interlanguage_links(arOut, 'ar.wikipedia.org/w/api.php' )
        #print(ILL)
        enILL = ILL['en'] #goes backward to the english ILL

        if arOut in out_ar_lyst_clean: #there is arILL in the arabic outlinks 
            enOutarILL.append(enILL)
            enOutarILL_arabic.append(arOut)

        else:
            onlyEn.append(arOut) #this is if it has an arILl but it isn't in the outlinks
            onlyEn_english.append(enILL)
    return onlyEn_english, enOutarILL

run functions

In [16]:
ILLdict = pd.DataFrame(columns = ['NoILL its only in English','only in English AS', 'shared'])#, index=outlinksUnique.values)
keys = outlinksUnique.keys()

In [17]:
for i in keys:
    lyst = [] #sets up list to add as a row

    onlyEnNoILL, arILLforEnOutlinks = seeIfnoILL(outlinksUnique[i])
    onlyEn_english, enOutarILL = seeRelatedtoAR(arILLforEnOutlinks, ar_everyOutlink)

    lyst = [len(onlyEnNoILL), len(onlyEn_english), len(enOutarILL)]
    ILLdict.loc[len(ILLdict)] = lyst #this does not bring over 

    

In [18]:
ILLdict.head()

,NoILL its only in English,only in English AS,shared
0,0,6,5
1,2,58,15
2,10,207,39
3,11,161,42
4,11,195,46


In [19]:
import pickle

with open('../Data/03b_ILLdf_en.pkl', 'wb') as f:
    pickle.dump(ILLdict, f)

### Arabic

make functions

In [20]:
#gets all of the english translations of the outlinks

def ar_seeIfnoILL(out_ar_lyst_clean):
    #if there is an english ILL
    enILLforArOutlinks = []
    #if there isn't one it is added to: 
    onlyArNoILL = [] 

    for arOut in out_ar_lyst_clean: #goes throguh all the arabic outlines
        ILL = wf.get_interlanguage_links(arOut, endpoint='ar.wikipedia.org/w/api.php')
        try:
            enILL = ILL['en']
            enILLforArOutlinks.append(enILL) #gets english ILL
            #print(enILL)
        except KeyError as ke:
            onlyArNoILL.append(arOut) #no english ILL 
            
    return onlyArNoILL, enILLforArOutlinks 



In [21]:
#if they are add it to: 


def seeRelatedtoEN(enILLforArOutlinks, out_en_lyst_clean):
    arOutenILL = []
    arOutenILL_english = []

    #if the english ILL is not referenced also in the arabic page add it to
    onlyAr = []
    onlyAr_arabic = []

    for enOut in enILLforArOutlinks: #goes through all the ones with ILLs in english
        ILL = wf.get_interlanguage_links(enOut, endpoint='ar.wikipedia.org/w/api.php')
        #print(enOut)
        arILL = ILL['ar'] #translates them back to arabic
        #print(arILL)

        if enOut in out_en_lyst_clean: #there is enILL in the english outlinks 
            arOutenILL.append(arILL) #adds the arabic version
            arOutenILL_english.append(enOut) #keept the english version

        else:
            onlyAr.append(enOut) #this is if it has an arILl but it isn't in the outlinks
            onlyAr_arabic.append(arILL)
            
    return onlyAr, arOutenILL #onlyEn_english, enOutarILL

run it

In [22]:
ar_ILLdict = pd.DataFrame(columns = ['NoILL its only in Arabic','only in Arabic AS', 'shared'])#, index=outlinksUnique.values)
ar_keys = ar_outlinksUnique.keys()

In [23]:
from tqdm import tqdm

onlyArNoILL_lyst = []
enILLforArOutlinks_lyst = []
onlyAr_lyst = []
arOutenILL_lyst = []

for i in tqdm(ar_keys):
    lyst = [] #sets up list to add as a row

    onlyArNoILL, enILLforArOutlinks  = ar_seeIfnoILL(ar_outlinksUnique[i])
    onlyAr, arOutenILL = seeRelatedtoEN(enILLforArOutlinks, everyOutlink)

    lyst = [len(onlyArNoILL), len(onlyAr), len(arOutenILL)]
    ar_ILLdict.loc[len(ar_ILLdict)] = lyst #this does not bring over 

  0%|          | 0/149 [00:00<?, ?it/s]

100%|██████████| 149/149 [3:15:16<00:00, 78.63s/it]  


In [24]:
ar_ILLdict

,NoILL its only in Arabic,only in Arabic AS,shared
0,1,0,11
1,11,33,50
2,10,44,47
3,10,48,47
4,12,86,56
...,...,...,...
144,29,136,107
145,29,137,107
146,29,138,107
147,29,138,107


In [25]:
with open('../Data/03b_ILLdf_ar.pkl', 'wb') as f:
    pickle.dump(ar_ILLdict, f)